In [1]:
import json
import numpy as np

import time
import xobjects as xo
import xtrack as xt
import xpart as xp

from tqdm import tqdm
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
from scipy import constants 
####################
# Choose a context #
####################

context = xo.ContextCpu()
#context = xo.ContextCpu(omp_num_threads=5)
#context = xo.ContextCupy()
#context = xo.ContextPyopencl('0.0')

buf = context.new_buffer()


# Ion properties:
m_u = 931.49410242e6 # eV/c^2 -- atomic mass unit
A = 207.98 # Lead-208
Z = 82  # Number of protons in the ion (Lead)
Ne = 3 # Number of remaining electrons (Lithium-like)
m_e = 0.511e6 # eV/c^2 -- electron mass
m_p = 938.272088e6 # eV/c^2 -- proton mass
c = 299792458.0 # m/s

m_ion = A*m_u + Ne*m_e # eV/c^2

equiv_proton_momentum = 236e9 # eV/c = gamma_p*m_p*v

gamma_p = np.sqrt( 1 + (equiv_proton_momentum/m_p)**2 ) # equvalent gamma for protons in the ring


p0c = equiv_proton_momentum*(Z-Ne) # eV/c
gamma = np.sqrt( 1 + (p0c/m_ion)**2 ) # ion relativistic factor
beta = np.sqrt(1-1/(gamma*gamma)) # ion beta




In [2]:
#%%
##################
# Build TrackJob #
##################

fname_sequence ='/home/pkruyt/cernbox/xsuite-laser/xtrack/test_data/sps_w_spacecharge/line_no_spacecharge_and_particle.json'



with open(fname_sequence, 'r') as fid:
     input_data = json.load(fid)
sequence = xt.Line.from_dict(input_data['line'])


n_part = int(1e6)

SPS_tracker = xt.Tracker(_context=context, _buffer=buf, line=sequence)

# Build a reference particle
particle_sample = xp.Particles(mass0=m_ion, q0=Z-Ne, p0c=p0c)

sigma_z = 22.5e-2
nemitt_x = 2e-6
nemitt_y = 2.5e-6


particles = xp.generate_matched_gaussian_bunch(
         num_particles=n_part,
         #total_intensity_particles=bunch_intensity,
         nemitt_x=nemitt_x, nemitt_y=nemitt_y, sigma_z=sigma_z,
         #R_matrix=r_matrix,
         particle_ref=particle_sample,
         tracker=SPS_tracker
         #,steps_r_matrix=steps_r_matrix
         )


sequence.particle_ref = particle_sample
twiss = sequence.twiss(symplectify=True)

twiss_dict=dict(twiss)



#%%
###################
# Linear Transfer #
###################

arc=xt.LinearTransferMatrix(Q_x=twiss['qx'], Q_y=twiss['qy'],
beta_x_0=twiss['betx'][0], beta_x_1=twiss['betx'][-1],
beta_y_0=twiss['bety'][0], beta_y_1=twiss['bety'][-1],
alpha_x_0=twiss['alfx'][0], alpha_x_1=twiss['alfx'][-1],
alpha_y_0=twiss['alfy'][0], alpha_y_1=twiss['alfy'][-1],
disp_x_0=twiss['dx'][0], disp_x_1=twiss['dx'][-1],
disp_y_0=twiss['dy'][0], disp_y_1=twiss['dy'][-1],
beta_s=twiss['betz0'],
Q_s=-twiss['qs'],
chroma_x=twiss['dqx'], chroma_y=twiss['dqy'])

SPS_lin = xt.Line()

SPS_lin.append_element(arc,'SPS_LinearTransferMatrix') 

Done loading line from dict.           
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
The argument tracker is deprecated. Please use line instead. <class 'DeprecationWarning'>
*** Maximum RMS bunch length 0.23592919520901076m.
... distance to target bunch length: -2.2500e-01
... distance to target bunch length: 4.4112e-03
... distance to target bunch length: 4.1449e-03
... distance to target bunch length: -3.3479e-03
... distance to target bunch length: 1.0467e-03
... distance to target bunch length: 2.0728e-04
... distance to target bunch length: -3.3833e-06
... distance to target bunch length: 4.9353e-08
... distance to target bunch length: -6.5924e-08
--> Bunch length: 0.2250000493531224
--> Emittance: 0.6606168164388687
Warning! scale_with_transverse_norm_emitt is deprecated. Use nemitt_x and nemitt_y instead.


In [3]:
##################
# Laser Cooler #
##################

sigma_dp = 2e-4 # relative ion momentum spread
#sigma_dp = std_delta # relative ion momentum spread


#laser-ion beam collision angle
theta_l = 2.6*np.pi/180 # rad
theta_l = 0
nx = 0; ny = -np.sin(theta_l); nz = -np.cos(theta_l)

# Ion excitation energy:
hw0 = 230.823 # eV
hc=constants.hbar*c/constants.e # eV*m (ħc)
lambda_0 = 2*np.pi*hc/hw0 # m -- ion excitation wavelength

lambda_l = lambda_0*gamma*(1 + beta*np.cos(theta_l)) # m -- laser wavelength

# Shift laser wavelength for fast longitudinal cooling:
#lambda_l = lambda_l*(1+1*sigma_dp) # m

laser_frequency = c/lambda_l # Hz
sigma_w = 2*np.pi*laser_frequency*sigma_dp
#sigma_w = 2*np.pi*laser_frequency*sigma_dp/2 # for fast longitudinal cooling


sigma_t = 1/sigma_w # sec -- Fourier-limited laser pulse
print('Laser pulse duration sigma_t = %.2f ps' % (sigma_t/1e-12))

print('Laser wavelength = %.2f nm' % (lambda_l/1e-9))

laser_waist_radius = 1.3e-3
#laser_waist_radius = 1.3e-7

laser_x=0.0020000
laser_x=0.000000

CWlaser = xt.CWLaser(_buffer=buf,
                      laser_x=laser_x,
                      
                      laser_direction_nx = 0,
                      laser_direction_ny = 0,
                      laser_direction_nz = -1,
                      laser_power        = 5e-3, # J
                      laser_wavelength = lambda_l, # m
                      laser_waist_radius = laser_waist_radius, # m
                      ion_excitation_energy = hw0, # eV
                      ion_excited_lifetime  = 76.6e-12, # sec
                          
   )
                          
   
SPS_lin.append_element(CWlaser, 'GammaFactory_IP')



Laser pulse duration sigma_t = 2.74 ps
Laser wavelength = 1033.86 nm


In [4]:
##################
# Tracking #
##################
x, px, delta, action_x = [], [], [], []
num_turns = int(1e0)
# loop over turns
for i in tqdm(range(num_turns)):
    
    x.append(particles.x)
    px.append(particles.px)
    delta.append(particles.delta)
    
    # track particle
    arc.track(particles)
    CWlaser.track(particles)

  0%|          | 0/1 [00:00<?, ?it/s]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...


bc2d613e81424e1681d5c24059166c57.c: In function ‘CWLaser_track_local_particle’:
bc2d613e81424e1681d5c24059166c57.c:3153:27: warning: implicit declaration of function ‘CWLaserData_get_laser_energy’; did you mean ‘CWLaserData_get_laser_power’? [-Wimplicit-function-declaration]
     double laser_energy = CWLaserData_get_laser_energy(el);
                           ^~~~~~~~~~~~~~~~~~~~~~~~~~~~
                           CWLaserData_get_laser_power
bc2d613e81424e1681d5c24059166c57.c:3167:37: warning: implicit declaration of function ‘CWLaserData_get_N_OmegaRabiTau_values’; did you mean ‘CWLaserData_get_laser_waist_radius’? [-Wimplicit-function-declaration]
     int64_t N_OmegaRabiTau_values = CWLaserData_get_N_OmegaRabiTau_values(el);
                                     ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                                     CWLaserData_get_laser_waist_radius
bc2d613e81424e1681d5c24059166c57.c:3168:41: warning: implicit declaration of function ‘CWLaserData_get_N_DeltaDet

Done compiling ContextCpu kernels.


ImportError: /home/pkruyt/cernbox/laser_cooling/bc2d613e81424e1681d5c24059166c57.cpython-311-x86_64-linux-gnu.so: undefined symbol: CWLaserData_get_OmegaRabiTau_max

In [ ]:
excited=particles.state==2

fraction_excitation = sum(excited)/len(excited)

plt.figure()
plt.title(f'Fraction of excited particles: {fraction_excitation*100}%')

plt.scatter(particles.x,particles.px)
plt.scatter(particles.x[excited],particles.px[excited])
plt.xlabel('x [m]')
plt.ylabel('px ')
plt.show()

In [ ]:
(particles.state==2)